In [1]:
import pandas as pd

from os.path import join

DATA_DIR = 'raw_data/'

## Connect DF

In [2]:
! head -6 raw_data/connect.csv

"cid","pid1","pid2","relation","location","locid","startdate","stopdate","addedby","dateadded","modby"
"2","2","3","1","University of California, Berkeley","312","2000-00-00","2005-00-00","david","2005-01-15 17:33:57","david"
"3","4","3","2","University of California, Berkeley","312","2003-00-00","2006-00-00","david","2005-01-15 17:35:17","david"
"5","6","3","1","University of California, Berkeley","312","2002-00-00","2008-00-00","david","2005-01-15 19:02:54","david"
"6","18761","9","1","University of California, Berkeley","312","0000-00-00","1984-00-00","david","2005-01-15 19:06:16","david"
"7","10","16","2","Washington University, Saint Louis","497","0000-00-00","0000-00-00","david","2005-01-15 19:06:58","david"


In [3]:
conn_df = pd.read_csv(join(DATA_DIR, 'connect.csv'), sep=',', dtype=str).drop(
    columns=['addedby','dateadded','modby']).fillna('')
conn_df.shape

(1553290, 8)

In [4]:
[(c, (conn_df[c] == '').sum()) for c in conn_df.columns]

[('cid', 0),
 ('pid1', 0),
 ('pid2', 0),
 ('relation', 0),
 ('location', 38958),
 ('locid', 0),
 ('startdate', 0),
 ('stopdate', 0)]

In [5]:
conn_df[(conn_df.location == '') & (conn_df.locid != '')].sample(5)

,cid,pid1,pid2,relation,location,locid,startdate,stopdate
1550888,1781404,838757,0,0,,1,0000-00-00,0000-00-00
1515163,1732522,824709,383257,1,,1,0000-00-00,0000-00-00
168986,198971,84269,0,0,,1,0000-00-00,0000-00-00
15282,18851,13705,4248,2,,1,0000-00-00,0000-00-00
1266620,1371007,721003,344580,1,,1,0000-00-00,0000-00-00


In [6]:
ds_people = pd.read_hdf('dataset/people_df.h5')
ds_people.shape

(774733, 13)

In [8]:
ds_people.head(1)

,pid,firstname,middlename,lastname,degrees,location,locid,majorarea,orcid,firstname_norm,middlename_norm,lastname_norm,orcid_norm
0,1,Stephen,V.,David,Ph.D.,Oregon Health and Science University,226,"neuro,csd,bme",0000-0003-4135-3104,STEPHEN,V,DAVID,0000-0003-4135-3104


In [9]:
pids = set(ds_people.pid)
len(pids)

774733

In [10]:
ds_conn = conn_df[(conn_df.pid1.isin(pids)) & (conn_df.pid2.isin(pids))].copy().reset_index(drop=True)
ds_conn.shape

(743490, 8)

In [11]:
ds_conn.cid.nunique()

743186

In [12]:
ds_conn.cid.value_counts()

112190     2
123561     2
117375     2
108957     2
117806     2
          ..
348727     1
927261     1
692492     1
1632993    1
777568     1
Name: cid, Length: 743186, dtype: int64

In [13]:
ds_conn[ds_conn.cid == '106829']

,cid,pid1,pid2,relation,location,locid,startdate,stopdate
110853,106829,83509,60134,1,University of Arkansas,1026,0000-00-00,2014-00-00
146208,106829,83509,60134,1,University of Arkansas,1026,0000-00-00,2014-00-00


In [14]:
ds_conn[ds_conn.cid == '114032']

,cid,pid1,pid2,relation,location,locid,startdate,stopdate
92806,114032,16179,6140,2,"Université de Montréal, and Centre de Recherch...",6236,2014-00-00,2016-00-00
118356,114032,16179,6140,2,"Université de Montréal, and Centre de Recherch...",6236,2014-00-00,2016-00-00


In [15]:
ds_conn = ds_conn.drop_duplicates().copy().reset_index(drop=True)
ds_conn.shape

(743196, 8)

In [16]:
ds_conn.cid.value_counts()

1396680    2
1568223    2
1702880    2
426298     2
1455639    2
          ..
409498     1
496757     1
1606834    1
61991      1
777568     1
Name: cid, Length: 743186, dtype: int64

In [17]:
ds_conn[ds_conn.cid == '1702880']

,cid,pid1,pid2,relation,location,locid,startdate,stopdate
712355,1702880,816978,351350,1,Texas A & M University,5832,2019-00-00,0000-00-00
713402,1702880,816977,817781,1,Rheinische Friedrich-Wilhelms-Universität Bonn,3182,0000-00-00,1999-00-00


In [18]:
dup_cids = set(k for k, v in ds_conn.cid.value_counts().items() if v > 1)
len(dup_cids)

10

In [19]:
ds_conn = ds_conn[~ds_conn.cid.isin(dup_cids)].copy().reset_index(drop=True)
ds_conn.shape

(743176, 8)

In [20]:
ds_conn.cid.nunique()

743176

In [21]:
len(set(ds_conn.pid1) | set(ds_conn.pid2))

738989

In [22]:
def norm_startdate(x):
    """"""
    parts = x.split('-')
    assert len(parts) == 3
    if parts[0] == '0000':
        return -1
    yr = int(parts[0])
    if yr < 1000 or yr > 2050:
        return -1
    return yr

ds_conn['startyear'] = ds_conn.startdate.apply(norm_startdate)
ds_conn.shape

(743176, 9)

In [23]:
ds_conn[ds_conn.startyear != -1].startyear.describe()

count    134960.000000
mean       2000.628416
std          28.194588
min        1015.000000
25%        1997.000000
50%        2007.000000
75%        2013.000000
max        2023.000000
Name: startyear, dtype: float64

In [24]:
def norm_stopdate(x):
    """"""
    parts = x.split('-')
    assert len(parts) == 3
    if parts[0] == '0000':
        return -1
    yr = int(parts[0])
    if yr > 2030:
        return -1
    return yr

ds_conn['stopyear'] = ds_conn.stopdate.apply(norm_stopdate)
ds_conn.shape

(743176, 10)

In [25]:
ds_conn[ds_conn.stopyear != -1].stopyear.describe()

count    611925.000000
mean       2002.177025
std          24.246215
min        1001.000000
25%        2002.000000
50%        2007.000000
75%        2011.000000
max        2030.000000
Name: stopyear, dtype: float64

In [26]:
ds_conn[ds_conn.stopyear != -1].stopyear.sort_values()

715183    1001
555797    1006
41050     1008
562833    1010
84146     1012
          ... 
734457    2026
734458    2026
9804      2026
734460    2026
120082    2030
Name: stopyear, Length: 611925, dtype: int64

In [27]:
ds_conn[ds_conn.stopyear == 2030]

,cid,pid1,pid2,relation,location,locid,startdate,stopdate,startyear,stopyear
120082,264293,92149,12374,4,Karolinska Institute,123,1960-00-00,2030-00-00,1960,2030


In [28]:
ds_conn.to_hdf('dataset/connect_df.h5', key='df', mode='w')